In [1]:
import folium
from property_finder.database import Database
import json
from datetime import datetime
import pandas as pd


with open('config.json') as json_data_file:
    config = json.load(json_data_file)
    
database = Database(database_path='data/database_old.db', reset_db=False)

locations = database.read_from_sql("""
SELECT Property.ID, Property.Price, Property.URL, Property.Bedrooms,
Location.Latitude, Location.Longitude, LSOA.MultipleDeprivationIndex,
Travel.JourneyDuration, Travel.JourneyFare 
FROM Property
LEFT JOIN Location
ON Property.ID = Location.ID
LEFT JOIN LSOA 
ON Property.ID = LSOA.ID
LEFT JOIN Travel
ON Property.ID = Travel.ID
WHERE Location.Longitude IS NOT NULL
AND LSOA.MultipleDeprivationIndex IS NOT NULL""")

locations.MultipleDeprivationIndex = locations.MultipleDeprivationIndex.astype(int)
locations['Color'] = pd.cut(locations.MultipleDeprivationIndex,
                            bins=4,
                            labels=['red', 'orange', 'yellow', 'green'])

locations.loc[locations.JourneyFare.isnull(), 'JourneyFare'] = 0
locations.loc[locations.JourneyDuration.isnull(), 'JourneyDuration'] = 0

In [2]:
def minutes_to_hhmm(minutes):
    mm = minutes % 60
    hh = int(minutes / 60)
    
    return '%dh %dmin' % (hh, mm)


def plot_map(locations, output_file='map.html', max_journey_time=None, min_mdi=None, min_bedrooms=None):
    if max_journey_time is not None:
        locations = locations[locations.JourneyDuration <= max_journey_time]
        
    if min_mdi is not None:
        locations = locations[locations.MultipleDeprivationIndex >= min_mdi]

    if min_bedrooms is not None:
        locations = locations[locations.Bedrooms >= min_bedrooms]
    
    m = folium.Map(location=[51.509865, -0.118092])
    tooltip_string = """Price: %f<br>
    Bedrooms: %d<br>
    MultipleDeprivationIndex: %d<br>
    JourneyDuration: %s <br>
    JourneyFare: %f"""


    for id_val, location in locations.iterrows():
        tooltip = tooltip_string % (location.Price, location.Bedrooms,
                                    location.MultipleDeprivationIndex,
                                    minutes_to_hhmm(location.JourneyDuration), 
                                    location.JourneyFare)
        popup = folium.Popup('<a href="%s" target="_blank">Rightmove webpage</a>' % location.URL)

        marker = folium.CircleMarker(location=[location.Latitude, location.Longitude],
                                     radius=7, color=location.Color, fill=location.Color,
                                     tooltip=tooltip, popup=popup)
        marker.add_to(m)

    m.save(output_file)

In [3]:
plot_map(locations)

In [4]:
plot_map(locations, output_file='map_filtered.html', max_journey_time=90, min_mdi=5000)